In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from astropy import constants as const
import astropy.units as u
import math
%matplotlib inline

Question 1 and 2 are meant to be by hand derivations and represented with LaTeX(see pdf)

Question 3: Solving the PRT equation

In [ ]:
from sympy import Function, Derivative
from sympy.abc import s # s is the independent variable
from sympy import symbols, Eq, Function
from sympy.solvers.ode.systems import dsolve_system
from sympy import *

In [ ]:
Q_v, U_v = symbols("Q_v U_v", cls=Function)
s, f_v = symbols("s , f_v") #defines path and Faraday rotation


In [ ]:
eqs = [Eq(Q_v(s).diff(s), (-f_v)*U_v(s)), Eq(U_v(s).diff(s), (f_v)*Q_v(s))] 
# defines the system of ODEs
init_printing()
eqs # shows the system

In [ ]:
# solving the system
# general solution
sol =dsolve_system(eqs)
print(sol)
init_printing()
sol # prints out the solution to the system

Note: My supervisor told me in person to use the initial conditions Q_v(0)=U_v(0)=0

In [ ]:
# solving the system
# particular solution for plotting using Q(0)=0 U(0)=1 
sol2=dsolve_system(eqs, ics={Q_v(0): 0, U_v(0): 1})
print(sol2)
init_printing(2)
sol2 # prints out the solution to the system


Question 4: Plotting the solutions in question 3


In [ ]:
# create a function that corresponds to a solution
f_v=1 # since it is constant, choose 1 for plotting 
func_Q = lambdify(s, -(sin(s)),'numpy') 
func_U = lambdify(s, cos(s),'numpy')
svals = np.arange(0,20,0.01)
yvals_Q=func_Q(svals)
yvals_U=func_U(svals)
    
    
# make figure
plt.plot(svals, yvals_Q)
plt.plot(svals, yvals_U)
plt.ylabel("Stokes parameters(erg $s^{−1}$ $cm^{−2}$ $Hz^{−1}$ $str^{−1}$)",size=13)
plt.xlabel("Path length(cm)", size=13)
plt.legend(['$Q_{v}(s)$', '$U_{v}(s)$' ], fontsize='13', loc= 'upper left')
plt.savefig('PRTPLOT.pdf')
plt.show()


Question 5: writing a function for Faraday rotation in the high frequency limit.

In [ ]:
# defining a function for thermal electron faraday rotation
def fth(v, n, B):
    """Function for faraday rotation with only thermal electrons present
    
    Parameters: 
    v-----frequency 
    n-----electron density
    B-----magnetic feild strength"
     
    Returns:
    faraday rotation with only thermal electrons present
    """
    # add units to the inputted values
    B_G = B*u.G
    v_Hz = v*u.s**-1
    n=n*u.cm**-3

    return (1/math.pi)*(((math.e)**3)/(((const.m_e.to(u.g))**2)*((const.c.to(u.cm/u.s))**4)))*n*B_G*(((const.c.to(u.cm/u.s))/v_Hz)**2)

Question 6: Evaluating Faraday Rotations at different frequencies. Since Bcostheta=1, B is the same as B along the line of sight. 

In [ ]:
# ISM 
fth(700e6,1e-1,10e-6) #700 MHz


In [ ]:
#ISM
fth(1.4e9,1e-1,10e-6)#1.4GHz

In [ ]:
#ICM
fth(700e6,1e-1,1e-6) #700 MHz

In [ ]:
#ICM
fth(1.4e9,1e-3,1e-6) #1.4 GHz

In [ ]:
#IGM
fth(700e6,1e-7,1e-9) #700MHz

In [ ]:
fth(1.4e9,1e-7,1e-9) #1.4GHz

In [ ]:
# defining a function for thermal electron faraday conversion
def hth(v, n, B):
    """Function for faraday rotation with only thermal electrons present
    
    Parameters: 
    v-----frequency 
    n-----electron density
    B-----magnetic feild strength"
     
    Returns:
    faraday rotation with only thermal electrons present
    """
    # add units to the inputted values
    B_G = B*u.G
    v_Hz = v*u.s**-1
    n=n*u .cm**-3

    return (1/(4*((math.pi)**2)))*(((math.e)**4)/(((const.m_e.to(u.g))**3)*((const.c.to(u.cm/u.s))**5)))*n*(B_G**2)*(((const.c.to(u.cm/u.s))/v_Hz)**3)

Question 7- Evaluting Faraday conversion (assuming sintheta=1), B is perpendicular to line of sight

In [ ]:
# ISM 
hth(700e6,1e-1,10e-6) #700 MHz


In [ ]:
# ISM
hth(1.4e9,1e-1,10e-6)#1.4GHz

In [ ]:
# ICM
hth(700e6,1e-1,1e-6) #700 MHz


In [ ]:
# ICM
hth(1.4e9,1e-3,1e-6) #1.4 GHz

In [ ]:
# IGM
hth(700e6,1e-7,1e-9) #700MHz

In [ ]:
# IGM
hth(1.4e9,1e-7,1e-9) #1.4GHz

Question 8 is a response answer(see pdf). Question 9 is a derivation done by hand and put into Latex(see pdf)

Question 10- Visualizing Data from the Green Bank Telescope

In [ ]:
from astropy.io import fits # importing relavant packages to analyse the data
from astropy.wcs import WCS
from astropy.utils.data import get_pkg_data_filename


In [ ]:
def plot_map(fn, s1, s2, s1_file, s2_file , clrlabel):
    """
    Creates maps for the inputted Stokes Parameter
    Parameters
    fn-----the data file
    s1-----the title(max)
    s2-----the title(min)
    s1_file---file name for max frequency
    s2_file---file name for min frequency
    clrlabel---title for color bar
    
    Returns:
    Colormap for the inputted  parameter
    """
    
    f = fits.open(fn)
    datacube=f[0].data

    selfreqind = 0 
    ax = plt.subplot(projection=wcs,  slices=('x', 'y', selfreqind ))
    ax.coords[2].set_ticklabel(exclude_overlapping=True)
    plt.title(s1)
    im=ax.imshow(datacube[selfreqind , :, :], cmap='RdBu_r')
    cbar = plt.colorbar(im)
    cbar.set_label(clrlabel, size=13)
    plt.savefig(s1_file)
    plt.show()
    
    #min frequency map
    selfreqind = -1
    ax = plt.subplot(projection=wcs, slices=('y', 'x', selfreqind )) 
    ax.coords[2].set_ticklabel(exclude_overlapping=True)
    plt.title(s2)
    im=ax.imshow(datacube[selfreqind , :, :], cmap='RdBu_r')
    cbar = plt.colorbar(im)
    plt.savefig(s2_file)

In [ ]:
# intensity plots
fn = get_pkg_data_filename('11hrCombinedCom20arcmin_I.fits') # open the data file
f = fits.open(fn)
wcs = WCS(f[0].header)
test = wcs.pixel_to_world
par1="Intensity map for maximum frequency"
par2="Intensity map for minimum frequency"
par1_file="Intensity map for maximum frequency.pdf"
par2_file="Intensity map for minimum frequency.pdf"
clr_lbl="I(erg $s^{−1}$ $cm^{−2}$ $Hz^{−1}$ $str^{−1}$)"
plot_map(fn, par1, par2, par1_file, par2_file,clr_lbl)


In [ ]:
# making Q parameter maps
fn_Q = get_pkg_data_filename('11hrCombinedCom20arcmin_Q_addedmeanQ.fits')
f_Q= fits.open(fn_Q)
wcs = WCS(f_Q[0].header)
par1="Q map for maximum frequency"
par2="Q map for minimum frequency"
par1_file="Q map for maximum frequency.pdf"
par2_file="Q map for minimum frequency.pdf"
clr_lbl="Q(erg $s^{−1}$ $cm^{−2}$ $Hz^{−1}$ $str^{−1}$)"
plot_map(fn_Q, par1, par2, par1_file, par2_file,clr_lbl)


In [ ]:
# making U maps 
fn_U = get_pkg_data_filename('11hrCombinedCom20arcmin_U_addedmeanU.fits')
f_U= fits.open(fn_U)
wcs = WCS(f_U[0].header)
par1="U map for maximum frequency"
par2="U map for minimum frequency"
par1_file="U map for maximum frequency.pdf"
par2_file="U map for minimum frequency.pdf"
clr_lbl="U(erg $s^{−1}$ $cm^{−2}$ $Hz^{−1}$ $str^{−1}$)"
plot_map(fn_U, par1, par2, par1_file, par2_file, clr_lbl)

In [ ]:
# making maps for  V
fn_V = get_pkg_data_filename('11hrCombinedCom20arcmin_V.fits')
f_V= fits.open(fn_V)
wcs = WCS(f_U[0].header)
par1="V map for maximum frequency"
par2="V map for minimum frequency"
par1_file="V map for maximum frequency.pdf"
par2_file="V map for minimum frequency.pdf"
clr_lbl="V(erg $s^{−1}$ $cm^{−2}$ $Hz^{−1}$ $str^{−1}$)"
plot_map(fn_V, par1, par2, par1_file, par2_file, clr_lbl)